# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [17]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
cities = pd.read_csv('../WeatherPy/output_data/cities.csv')
cities.dropna(inplace=True, how='any')
cities


,Unnamed: 0,city,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
0,0,Ketchikan,55.3422,-131.6461,50.47,50.47,19.57,90,87
1,1,Cape Town,-33.9258,18.4232,59.31,61.27,3.00,75,82
2,3,Makkah al Mukarramah,21.4267,39.8261,85.06,85.06,4.54,6,46
3,4,Port Alfred,-33.5906,26.8910,61.41,61.41,2.39,29,85
4,5,Souillac,-20.5167,57.5167,73.78,75.36,3.44,75,88
...,...,...,...,...,...,...,...,...,...
539,597,Partenit,44.5780,34.3446,57.20,57.20,9.24,13,66
540,598,Tocopilla,-22.0920,-70.1979,64.36,64.36,7.14,21,61
541,599,Beloha,-25.1667,45.0500,65.73,65.73,11.74,100,74
542,600,Bandarbeyla,9.4942,50.8122,77.92,77.92,7.99,96,77


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Configure gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '275px',
    'height': '275px',
    'border': '1px solid black',
    'padding': '1px',
    'zoom_level': 12
}

# Use the Lat and Lng as locations and Humidity as the weight
locations = cities[['latitude', 'longitude']]

humidity = cities['humidity']

# Add Heatmap layer to map
map = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10,
                                point_radius=1)

map.add_layer(heat_layer)

map


Figure(layout=FigureLayout(border='1px solid black', height='275px', padding='1px', width='275px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
ideal_cities = cities.loc[(cities['max temperature'] < 80) & (cities['max temperature'] > 70) & (cities['humidity'] <30)]
ideal_cities

,Unnamed: 0,city,latitude,longitude,temperature,max temperature,wind speed,cloudiness,humidity
24,29,Adrar,20.5022,-10.0711,78.10,78.10,8.99,1,21
84,95,Jerome,42.7241,-114.5186,71.67,72.43,14.97,1,27
130,148,Alice Springs,-23.7000,133.8833,74.75,74.75,9.22,0,23
134,152,Prescott Valley,34.5894,-112.3252,73.13,75.96,3.00,1,23
135,153,Aswān,24.0934,32.9070,78.04,78.04,14.09,0,26
177,205,Dutlwe,-23.9833,23.9000,72.86,72.86,7.78,0,14
243,274,Comodoro Rivadavia,-45.8667,-67.5000,71.46,71.46,26.46,0,13
459,507,Nālūt,30.3333,10.8500,76.71,76.71,11.05,7,27
497,553,Villazón,-22.0866,-65.5942,74.98,74.98,28.07,65,3
513,569,Turbat,26.0023,63.0440,76.41,76.41,7.52,0,17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = ideal_cities[['city', 'latitude', 'longitude']]
hotel_df['Hotel Name'] = ''
hotel_df.reset_index(drop=True)

<ipython-input-24-e6d4478e7f60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,city,latitude,longitude,Hotel Name
0,Adrar,20.5022,-10.0711,
1,Jerome,42.7241,-114.5186,
2,Alice Springs,-23.7000,133.8833,
3,Prescott Valley,34.5894,-112.3252,
4,Aswān,24.0934,32.9070,
5,Dutlwe,-23.9833,23.9000,
6,Comodoro Rivadavia,-45.8667,-67.5000,
7,Nālūt,30.3333,10.8500,
8,Villazón,-22.0866,-65.5942,
9,Turbat,26.0023,63.0440,


In [32]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "location": f'37.6775, -113.0619',
    "radius": 5000,
    "keyword": 'hotel',
    "key": g_key
}

response = requests.get(base_url, params=params).json()
    
results = response['results']

In [33]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": 'lodging',
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row['latitude']
    lng = row['longitude']
    city = row['city']

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'
       

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    try:
        
        print(f'Getting result for {city}')
        # add the results to the dataframe
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

# check the results
hotel_df

Getting result for Adrar
Missing field/result... skipping.
Getting result for Jerome
Missing field/result... skipping.
Getting result for Alice Springs
Missing field/result... skipping.
Getting result for Prescott Valley
Missing field/result... skipping.
Getting result for Aswān
Missing field/result... skipping.
Getting result for Dutlwe
Missing field/result... skipping.
Getting result for Comodoro Rivadavia
Missing field/result... skipping.
Getting result for Nālūt
Missing field/result... skipping.
Getting result for Villazón
Missing field/result... skipping.
Getting result for Turbat
Missing field/result... skipping.


,city,latitude,longitude,Hotel Name
24,Adrar,20.5022,-10.0711,
84,Jerome,42.7241,-114.5186,
130,Alice Springs,-23.7000,133.8833,
134,Prescott Valley,34.5894,-112.3252,
135,Aswān,24.0934,32.9070,
177,Dutlwe,-23.9833,23.9000,
243,Comodoro Rivadavia,-45.8667,-67.5000,
459,Nālūt,30.3333,10.8500,
497,Villazón,-22.0866,-65.5942,
513,Turbat,26.0023,63.0440,


In [34]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [35]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '275px',
    'height': '275px',
    'border': '1px solid black',
    'padding': '1px',
    'zoom_level': 12
}
fig = gmaps.figure(layout=figure_layout)

# Display figure
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(border='1px solid black', height='275px', padding='1px', width='275px'))